In [8]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import os 
import warnings
import catboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_curve
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import standardScaler

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
df = pd.read_csv('data/high_diamond_ranked_10min.csv')
display(df.head(5), df.info())

In [ ]:
info.columns

In [ ]:
info.blueWins.value_counts(normalize=True)

In [ ]:
plt.bar(info.blueWins.unique(),info.blueWins.value_counts(),color=("r","b")tick_label=("Loss","Win"))

In [ ]:
plt.scatter(info.blueAssists,info.redAssists,c=info.blueWins)
plt.xlabel("blue assists")
plt.ylabel("red assists")

In [ ]:
info.blueTotalGold.where(info.blueWins==0).sort_values(ascending=False).head(2)

In [ ]:
plt.scatter(data.blueTotalGold,data.redTotalGold,c=data.blueWins)
plt.vlines(21056,12000,24000,color= "red", linestyle=':')
plt.vlines(20000,12000,24000,color= "red", linestyle=':')
plt.axvspan(21100, 24000, color='g', alpha=0.5, lw=0)
plt.xlabel("blue gold")
plt.ylabel("red gold")

In [ ]:
print(info.blueGoldDiff.where(info.blueWins==0).sort_values(ascending=False).head(1))
print(info.blueGoldDiff.where(info.blueWins==1).sort_values(ascending=True).head(1))

In [ ]:
plt.scatter(data.blueGoldDiff,data.blueWins,c=data.blueWins)
plt.axvspan(-6324.0,6744.0,color='r',alpha=0.5,lw=0)
plt.ylabel("Lose                                             Win")
plt.xlabel("team gold")

In [ ]:
plt.scatter(info.blueWardsPlaced,info.redWardsPlaced,c=info.blueWins)
plt.xlabel("blue vision")
plt.ylabel("red vision")

In [ ]:
info.columns


In [ ]:
corr_matrix=info.drop(["gameId",'redWardsPlaced', 'redWardsDestroyed',
       'redFirstBlood', 'redKills', 'redDeaths', 'redAssists',
       'redEliteMonsters', 'redDragons', 'redHeralds', 'redTowersDestroyed',
       'redTotalGold', 'redAvgLevel', 'redTotalExperience',
       'redTotalMinionsKilled', 'redTotalJungleMinionsKilled', 'redGoldDiff',
       'redExperienceDiff', 'redCSPerMin', 'redGoldPerMin'],axis=1).corr(method='pearson')
corr_matrix.iloc[:,0]

In [ ]:
data.drop(["gameId",'redWardsPlaced', 'redWardsDestroyed',
       'redFirstBlood', 'redKills', 'redDeaths', 'redAssists',
       'redEliteMonsters', 'redDragons', 'redHeralds', 'redTowersDestroyed',
       'redTotalGold', 'redAvgLevel', 'redTotalExperience',
       'redTotalMinionsKilled', 'redTotalJungleMinionsKilled', 'redGoldDiff',
       'redExperienceDiff', 'redCSPerMin', 'redGoldPerMin'],axis=1).describe()

In [ ]:
data.blueWins.where(((data.blueKills+data.blueAssists)/data.blueDeaths)>=3).dropna().value_counts(normalize=True)


In [ ]:
data.blueWins.where(data.blueTowersDestroyed>=1).dropna().value_counts(normalize=True)


In [ ]:
data.blueWins.where(data.blueKills>=8).dropna().value_counts(normalize=True)


In [ ]:
info["blueKDA"]=3
for i in range(0,len(info)):
    if data.blueDeaths[i] != 0:
        info["blueKDA"][i]=((data.blueKills[i]+data.blueAssists[i])/data.blueDeaths[i]).round(4)
    else:
        0

In [ ]:
info["redKDA"]=3
for i in range(0,len(info)):
    if data.redDeaths[i]!=0:
        info["redKDA"][i] = ((data.redKills[i]+data.redAssists[i])/data.redDeaths[i]).round(4)
    else:
        0

In [ ]:
info["blueKDADiff"]=info["blueKDA"]-info["redKDA"]


In [ ]:
info["blueGoldAdv"]=3
for i in range(0,len(info)):
    if info["blueTotalGold"][i] >= info["blueTotalGold"][3464]:
        info["blueGoldAdv"][i] = 1
    else:
        info["blueGoldAdv"][i] = 0


In [ ]:
info["blueDiffNeg"]=3
for i in range(0,len(info)):
    if info["blueGoldDiff"][i] <= info["blueGoldDiff"][8459]:
        info["blueDiffNeg"][i] = 1
    else:
        info["blueDiffNeg"][i] = 0


In [ ]:
info["blueDiffPos"]=3
for i in range(0,9879):
    if info["blueGoldDiff"][i] >= info["blueGoldDiff"][9608]:
        info["blueDiffPos"][i] = 1
    else:
        info["blueDiffPos"][i] = 0

In [ ]:
cols=['gameId', 'blueWins', 'blueWardsPlaced', 'blueWardsDestroyed',
       'blueFirstBlood', 'blueKills', 'blueDeaths', 'blueAssists',
       'blueEliteMonsters', 'blueDragons', 'blueHeralds',
       'blueTowersDestroyed', 'blueTotalGold', 'blueAvgLevel',
       'blueTotalExperience', 'blueTotalMinionsKilled',
       'blueTotalJungleMinionsKilled', 'blueGoldDiff', 'blueExperienceDiff',
       'blueCSPerMin', 'blueGoldPerMin']
info[cols]

In [ ]:
info_x=info.drop(["gameId","blueWins"],axis=1)
info_y=info.blueWins

knn


In [ ]:
cv=KFold(n_splits = 5, random_state=2022,shuffle=True)


In [ ]:
np.sqrt(len(info_x))


In [ ]:
knn_params = {"n_neighbors" :np.arange(95,105),
             "weights" : ["distance"],
             "algorithm" : ["ball_tree"] , #,"kd_tree","brute"
             "leaf_size":[1,2]}


In [ ]:
knn = KNeighborsClassifier()
grid_knn = GridSearchCV(knn, knn_params,cv=5,verbose=2,n_jobs=-1)
grid_knn.fit(info_x,info_y)


In [ ]:
knn_best_params=grid_knn.best_params_
knn_best_params

In [ ]:
knn = KNeighborsClassifier(algorithm="ball_tree",n_neighbors=98,weights="distance",leaf_size=1)


In [ ]:
  np.mean(cross_val_score(knn,info_x,info_y,cv=cv,scoring="accuracy",n_jobs=-1))


In [ ]:
log_params = {"penalty":["l1","l2","elasticnet"],
              "solver":['newton-cg', 'lbfgs', 'sag', 'saga'],
             "C":[1,5,10,20,30,50,100],
             "warm_start":[True,False]}

In [ ]:
logreg=LogisticRegression()


In [ ]:
log_grid = GridSearchCV(logreg,log_params,cv=cv,verbose=2,n_jobs=-1)


In [ ]:
log_grid.best_params_


In [ ]:
logreg=LogisticRegression(penalty="l2",solver="sag",C=50, warm_start=False)


In [ ]:
cat_params = {"learning_rate":[0.005],
              "depth":[6,8]}

In [ ]:
cat = CatBoostClassifier()


In [ ]:
cat_grid = GridSearchCV(cat,cat_params,cv=cv)


In [ ]:
cat_grid.fit(info_x,info_y)
